In [18]:
import os
import math
from PIL import Image
import pandas as pd
import torch
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision.models.detection import retinanet_resnet50_fpn_v2
import torchvision.transforms as T


In [19]:
class LunaPatchRetinaDataset(torch.utils.data.Dataset):
    """
    Dataset that:
    - reads patch images from patches/
    - reads true nodules from annotations.csv
    - matches by seriesuid + 3D distance
    - returns (image, target) for RetinaNet
    """
    def __init__(self, patches_dir="patches", annotations_csv="annotations.csv", transforms=None):
        self.patches_dir = patches_dir
        self.transforms = transforms

        # 1) list all patch images
        self.img_paths = sorted(
            [
                os.path.join(patches_dir, f)
                for f in os.listdir(patches_dir)
                if f.lower().endswith(".tiff") or f.lower().endswith(".tif")
            ]
        )

        if len(self.img_paths) == 0:
            raise RuntimeError(f"No .tiff patches found in {patches_dir}")

        # 2) load LUNA16 annotations
        # IMPORTANT: in your folder it's just "annotations.csv"
        df = pd.read_csv(annotations_csv)

        # 3) group nodules by seriesuid
        self.nodules_by_uid = {}
        for _, row in df.iterrows():
            uid = str(row["seriesuid"])
            self.nodules_by_uid.setdefault(uid, []).append(
                {
                    "x": float(row["coordX"]),
                    "y": float(row["coordY"]),
                    "z": float(row["coordZ"]),
                    "d": float(row["diameter_mm"]),
                }
            )

    def __len__(self):
        return len(self.img_paths)

    def _parse_filename(self, path):
        """
        Expecting: patch_<uid>_<x>_<y>_<z>.tiff
        """
        name = os.path.basename(path)
        # drop extension
        if name.lower().endswith(".tiff"):
            name = name[:-5]
        else:
            name = os.path.splitext(name)[0]

        parts = name.split("_", 4)
        if len(parts) != 5:
            raise ValueError(f"Unexpected patch name format: {name}")
        _, uid, wx, wy, wz = parts
        return uid, float(wx), float(wy), float(wz)

    def _match_nodule(self, uid, wx, wy, wz):
        """
        Return matched nodule dict if candidate is close enough.
        """
        nods = self.nodules_by_uid.get(uid, [])
        for n in nods:
            dx = wx - n["x"]
            dy = wy - n["y"]
            dz = wz - n["z"]
            dist = math.sqrt(dx*dx + dy*dy + dz*dz)
            if dist <= n["d"] / 2.0:
                return n
        return None

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        uid, wx, wy, wz = self._parse_filename(img_path)

        # load image
        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        match = self._match_nodule(uid, wx, wy, wz)

        if match is not None:
            # positive example → put 1 box in center
            approx_pix = int(round(match["d"]))  # crude mm→px
            box_size = max(8, min(approx_pix, min(w, h) - 2))
            cx, cy = w / 2.0, h / 2.0
            x1 = cx - box_size / 2.0
            y1 = cy - box_size / 2.0
            x2 = cx + box_size / 2.0
            y2 = cy + box_size / 2.0
            boxes = torch.tensor([[x1, y1, x2, y2]], dtype=torch.float32)
            labels = torch.tensor([1], dtype=torch.int64)
        else:
            # negative example → no boxes
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
        }

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target


In [20]:
def collate_fn(batch):
    return tuple(zip(*batch))


In [21]:
def get_model(num_classes=2):
    # For torchvision >= 0.14 style
    model = retinanet_resnet50_fpn_v2(
        weights=None,          # start from scratch
        num_classes=num_classes
    )
    return model

def train_one_epoch(model, optimizer, dataloader, device):
    model.train()
    total_loss = 0.0
    for images, targets in dataloader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        loss = sum(loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


In [22]:
def main():
    patches_dir = "patches"
    annotations_csv = "annotations.csv"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    transforms = T.Compose([T.ToTensor()])

    dataset = LunaPatchRetinaDataset(
        patches_dir=patches_dir,
        annotations_csv=annotations_csv,
        transforms=transforms,
    )

    val_ratio = 0.1
    n_total = len(dataset)
    n_val = int(n_total * val_ratio)
    n_train = n_total - n_val
    train_ds, val_ds = random_split(dataset, [n_train, n_val])

    train_loader = DataLoader(
        train_ds,
        batch_size=8,
        shuffle=True,
        num_workers=0,      # ← important on Mac / notebook
        collate_fn=collate_fn,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=8,
        shuffle=False,
        num_workers=0,      # ← important
        collate_fn=collate_fn,
    )

    model = get_model(num_classes=2).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

    epochs = 5
    for epoch in range(epochs):
        loss = train_one_epoch(model, optimizer, train_loader, device)
        print(f"Epoch {epoch+1}/{epochs} - train loss: {loss:.4f}")

        model.eval()
        with torch.no_grad():
            for images, targets in val_loader:
                images = [img.to(device) for img in images]
                preds = model(images)
                break
        print("Ran validation forward pass ✅")

    torch.save(model.state_dict(), "retinanet_luna_patches.pth")
    print("Model saved to retinanet_luna_patches.pth")


In [23]:
#if __name__ == "__main__":
   # main()


In [24]:
import os, pandas as pd

print("cwd:", os.getcwd())
print("here:", os.listdir("."))

df = pd.read_csv("annotations.csv")
print("annotations shape:", df.shape)

print("patches:", len(os.listdir("patches")))
print(os.listdir("patches")[:5])


cwd: /Users/andy/Documents/Schoolwork/2025-2026/BMME 575/ML_project
here: ['RetinaNet.ipynb', '.DS_Store', 'Preprocessor.ipynb', 'patches', 'SimpleITK Tutorial(1)(1).ipynb', '.ipynb_checkpoints', 'annotations.csv', 'subset0', 'candidates_V2.csv']
annotations shape: (1186, 5)
patches: 79135
['patch_1.3.6.1.4.1.14519.5.2.1.6279.6001.210837812047373739447725050963_-263.43_-80.75_-93.55.tiff', 'patch_1.3.6.1.4.1.14519.5.2.1.6279.6001.657775098760536289051744981056_-161.22_58.23_69.25.tiff', 'patch_1.3.6.1.4.1.14519.5.2.1.6279.6001.716498695101447665580610403574_-181.19_-112.83_47.07.tiff', 'patch_1.3.6.1.4.1.14519.5.2.1.6279.6001.525937963993475482158828421281_-72.02_57.39_47.44.tiff', 'patch_1.3.6.1.4.1.14519.5.2.1.6279.6001.194440094986948071643661798326_-98.80_-165.46_-64.19.tiff']


In [25]:
transforms = T.Compose([T.ToTensor()])
ds = LunaPatchRetinaDataset(
    patches_dir="patches",
    annotations_csv="annotations.csv",
    transforms=transforms,
)

img, target = ds[0]
print("img shape:", img.shape)
print("target:", target)


img shape: torch.Size([3, 65, 65])
target: {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64), 'image_id': tensor([0])}


In [26]:
from torch.utils.data import DataLoader

loader = DataLoader(
    ds,
    batch_size=4,
    shuffle=False,
    num_workers=0,     # <- keep 0 on Mac/notebook
    collate_fn=collate_fn,
)

batch = next(iter(loader))
images, targets = batch
print(len(images), len(targets))
print(images[0].shape, targets[0])


4 4
torch.Size([3, 65, 65]) {'boxes': tensor([], size=(0, 4)), 'labels': tensor([], dtype=torch.int64), 'image_id': tensor([0])}


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = get_model(num_classes=2).to(device)

images, targets = next(iter(loader))   # from your dataloader with num_workers=0
images = [img.to(device) for img in images]
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

model.train()
loss_dict = model(images, targets)
loss = sum(loss_dict.values())
print(loss_dict)
print("total loss:", loss.item())


{'classification': tensor(0.3882, grad_fn=<DivBackward0>), 'bbox_regression': tensor(0., grad_fn=<DivBackward0>)}
total loss: 0.38817182183265686


In [28]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

optimizer.zero_grad()
loss.backward()
optimizer.step()

print("one training step ✅")


one training step ✅
